# Objectives
- Read CSV, JSON, and XML file types
- Extract the required data from the different file types. 
- Transform data to the required format. 
- Save the transformed data in a ready-to-load format, which can be loaded into an RDBMS.

# Download and unzip file in local directory

In [61]:
import urllib.request
import zipfile

# Define the URL and the local filename
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip"
local_filename = "source.zip"

# Download the file from the URL
urllib.request.urlretrieve(url, local_filename)

# Unzip the file
with zipfile.ZipFile(local_filename, 'r') as zip_ref:
    zip_ref.extractall("extracted_files")



In [62]:
import glob 
import pandas as pd 
import xml.etree.ElementTree as ET 
from datetime import datetime 
import os

#Global variables used by various functions
log_file = "log_file.txt" #store all logs
target_file = "transformed_data.csv" #Store final output data that can be loaded to a database 

# EXTRACT

### CSV Extraction Method

In [63]:
def extract_from_csv(file_to_process): 
    dataframe = pd.read_csv(file_to_process) 
    return dataframe 

### JSON Extraction Method

In [64]:
def extract_from_json(file_to_process): 
    dataframe = pd.read_json(file_to_process, lines=True) #If we explore the json file we can see that each new-line is an json object. So, this parameter indiicates a newline-delimter. 
    return dataframe 

### XML Extraction Method

In [65]:
def extract_from_xml(file_to_process): 
    dataframe = pd.DataFrame(columns=["name", "height", "weight"]) 
    tree = ET.parse(file_to_process) 
    root = tree.getroot() 
    for person in root: 
        name = person.find("name").text 
        height = float(person.find("height").text) 
        weight = float(person.find("weight").text) 
        dataframe = pd.concat([dataframe, pd.DataFrame([{"name":name, "height":height, "weight":weight}])], ignore_index=True) 
    return dataframe 

### Extract all Data Function

In [66]:
def extract(directory):
    extracted_data = pd.DataFrame(columns=['name', 'height', 'weight'])  # create an empty data frame to hold extracted data
    
    # Process all CSV files in the directory
    for csvfile in glob.glob(os.path.join(directory, "*.csv")):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index=True)
    
    # Process all JSON files in the directory
    for jsonfile in glob.glob(os.path.join(directory, "*.json")):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_json(jsonfile))], ignore_index=True)
    
    # Process all XML files in the directory
    for xmlfile in glob.glob(os.path.join(directory, "*.xml")):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_xml(xmlfile))], ignore_index=True)
    
    return extracted_data

# TRANSFORM


In [67]:
#We are interested in working with international units so pounds will go to kilgram and inches to centimeters.
def transform(data): 
    '''Convert inches to meters and round off to two decimals 
    1 inch is 0.0254 meters '''
    data['height'] = round(data.height * 0.0254,2) 
 
    '''Convert pounds to kilograms and round off to two decimals 
    1 pound is 0.45359237 kilograms '''
    data['weight'] = round(data.weight * 0.45359237,2) 
    
    return data 

# LOADING

#### Load data to a csv file

In [68]:
def load_data(target_file, transformed_data): 
    transformed_data.to_csv(target_file) 

#### Logging Operations

In [69]:
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open(log_file,"a") as f: 
        f.write(timestamp + ',' + message + '\n') 

# TESTING ETL PROCESS

In [70]:
# Log the initialization of the ETL process 
log_progress("ETL Job Started") 
 
# Log the beginning of the Extraction process 
log_progress("Extract phase Started") 
extracted_data = extract('extracted_files') 
 
# Log the completion of the Extraction process 
log_progress("Extract phase Ended") 
 
# Log the beginning of the Transformation process 
log_progress("Transform phase Started") 
transformed_data = transform(extracted_data) 
print("Transformed Data") 
print(transformed_data) 
 
# Log the completion of the Transformation process 
log_progress("Transform phase Ended") 
 
# Log the beginning of the Loading process 
log_progress("Load phase Started") 
load_data(target_file,transformed_data) 
 
# Log the completion of the Loading process 
log_progress("Load phase Ended") 
 
# Log the completion of the ETL process 
log_progress("ETL Job Ended") 

Transformed Data
     name  height  weight
0    alex    1.67   51.25
1    ajay    1.82   61.91
2   alice    1.76   69.41
3    ravi    1.73   64.56
4     joe    1.72   65.45
5    alex    1.67   51.25
6    ajay    1.82   61.91
7   alice    1.76   69.41
8    ravi    1.73   64.56
9     joe    1.72   65.45
10   alex    1.67   51.25
11   ajay    1.82   61.91
12  alice    1.76   69.41
13   ravi    1.73   64.56
14    joe    1.72   65.45
15   jack    1.74   55.93
16    tom    1.77   64.18
17  tracy    1.78   61.90
18   john    1.72   50.97
19   jack    1.74   55.93
20    tom    1.77   64.18
21  tracy    1.78   61.90
22   john    1.72   50.97
23   jack    1.74   55.93
24    tom    1.77   64.18
25  tracy    1.78   61.90
26   john    1.72   50.97
27  simon    1.72   50.97
28  jacob    1.70   54.73
29  cindy    1.69   57.81
30   ivan    1.72   51.77
31  simon    1.72   50.97
32  jacob    1.70   54.73
33  cindy    1.69   57.81
34   ivan    1.72   51.77
35  simon    1.72   50.97
36  jacob    1.70   5